In [ ]:
# Copier dans un fichier notebook nommé ml-vs-dl-comparison.ipynb
# Notebook de comparaison: Machine Learning classique vs Deep Learning
# BTS SIO - Séance 1: Découverte des concepts

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import time
import seaborn as sns

# Configuration pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print("# Comparaison Machine Learning classique vs Deep Learning")
print("Ce notebook vous permet d'observer concrètement les différences entre ces deux approches.")

# Partie 1: Chargement des données MNIST
print("\n## 1. Chargement du dataset MNIST")
print("Chargement du jeu de données de chiffres manuscrits...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f"Forme des données d'entraînement: {x_train.shape}")
print(f"Forme des données de test: {x_test.shape}")
print(f"Nombre de classes: {len(np.unique(y_train))}")

# Afficher quelques exemples d'images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f"Chiffre: {y_train[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# Partie 2: Machine Learning classique avec Random Forest
print("\n## 2. Approche Machine Learning classique (Random Forest)")

# Prétraitement pour Machine Learning classique
print("\n### 2.1 Prétraitement pour Machine Learning classique")
print("Pour le ML classique, nous devons transformer les images 2D en vecteurs 1D et réduire leur dimensionnalité.")

# Nous utiliserons un sous-ensemble pour l'efficacité
n_samples = 10000
x_train_subset = x_train[:n_samples]
y_train_subset = y_train[:n_samples]

# Aplatir les images 28x28 en vecteurs de 784 pixels
X_train_flat = x_train_subset.reshape(n_samples, -1)
X_test_flat = x_test.reshape(x_test.shape[0], -1)

print(f"Forme après aplatissement: {X_train_flat.shape}")

# Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)
print("Données normalisées avec StandardScaler")

# Réduction de dimensionnalité avec PCA
pca = PCA(n_components=50)  # Réduire à 50 dimensions
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Forme après PCA: {X_train_pca.shape}")
print(f"Variance expliquée: {np.sum(pca.explained_variance_ratio_)*100:.2f}%")

# Entraînement du modèle Random Forest
print("\n### 2.2 Entraînement du modèle Random Forest")

start_time = time.time()
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_pca, y_train_subset)
ml_training_time = time.time() - start_time

print(f"Temps d'entraînement: {ml_training_time:.2f} secondes")

# Prédictions et évaluation
y_pred_rf = rf.predict(X_test_pca)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Précision du Random Forest: {accuracy_rf*100:.2f}%")

# Partie 3: Deep Learning avec CNN
print("\n## 3. Approche Deep Learning (CNN)")

# Prétraitement pour Deep Learning
print("\n### 3.1 Prétraitement pour Deep Learning")
print("Pour le Deep Learning, le prétraitement est plus simple, nous normalisons simplement les pixels.")

# Simplement normaliser les pixels entre 0 et 1
X_train_dl = x_train_subset.reshape(n_samples, 28, 28, 1) / 255.0
X_test_dl = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255.0

print(f"Forme pour le CNN: {X_train_dl.shape}")
print("Données normalisées par simple division (pixels entre 0 et 1)")

# Création d'un modèle CNN simple
print("\n### 3.2 Création et entraînement d'un modèle CNN")

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Architecture du modèle CNN:")
model.summary()

# Entraînement rapide (juste quelques époques pour la démonstration)
start_time = time.time()
history = model.fit(
    X_train_dl, y_train_subset,
    epochs=3,
    validation_split=0.2,
    verbose=1
)
dl_training_time = time.time() - start_time

# Évaluation
loss, accuracy_dl = model.evaluate(X_test_dl, y_test, verbose=0)
print(f"\nTemps d'entraînement: {dl_training_time:.2f} secondes")
print(f"Précision du modèle CNN: {accuracy_dl*100:.2f}%")

# Partie 4: Test de généralisation
print("\n## 4. Test de généralisation (robustesse au bruit)")

# Créer des versions perturbées du jeu de test
def add_noise(images, noise_level=0.3):
    """Ajoute du bruit gaussien aux images"""
    noisy_images = images.copy()
    
    if len(images.shape) == 4:  # Pour le CNN (avec canal)
        noise = np.random.normal(0, noise_level, images.shape)
        noisy_images += noise
        return np.clip(noisy_images, 0, 1)
    else:  # Pour le ML classique (sans canal)
        noise = np.random.normal(0, noise_level*255, images.shape)
        noisy_images += noise
        return np.clip(noisy_images, 0, 255)

# Créer des images de test bruitées
n_test_examples = 1000
X_test_subset = X_test_pca[:n_test_examples]
X_test_dl_subset = X_test_dl[:n_test_examples]
y_test_subset = y_test[:n_test_examples]

# Images bruitées pour ML classique
X_test_flat_subset = X_test_flat[:n_test_examples]
X_test_noisy_flat = add_noise(X_test_flat_subset.reshape(-1, 28, 28), 0.3).reshape(-1, 784)
X_test_noisy_pca = pca.transform(scaler.transform(X_test_noisy_flat))

# Images bruitées pour Deep Learning
X_test_noisy_dl = add_noise(X_test_dl_subset, 0.3)

# Évaluer les deux modèles sur les données bruitées
y_pred_rf_noisy = rf.predict(X_test_noisy_pca)
accuracy_rf_noisy = accuracy_score(y_test_subset, y_pred_rf_noisy)

loss_noisy, accuracy_dl_noisy = model.evaluate(X_test_noisy_dl, y_test_subset, verbose=0)

print(f"Précision du Random Forest sur données normales: {accuracy_rf*100:.2f}%")
print(f"Précision du Random Forest sur données bruitées: {accuracy_rf_noisy*100:.2f}%")
print(f"Baisse de performance: {(accuracy_rf - accuracy_rf_noisy)*100:.2f}%")

print(f"\nPrécision du CNN sur données normales: {accuracy_dl*100:.2f}%")
print(f"Précision du CNN sur données bruitées: {accuracy_dl_noisy*100:.2f}%")
print(f"Baisse de performance: {(accuracy_dl - accuracy_dl_noisy)*100:.2f}%")

# Visualiser quelques exemples
plt.figure(figsize=(12, 6))
for i in range(5):
    # Image originale
    plt.subplot(2, 5, i+1)
    plt.imshow(X_test_dl_subset[i].reshape(28, 28), cmap='gray')
    plt.title(f"Original: {y_test_subset[i]}")
    plt.axis('off')
    
    # Image bruitée
    plt.subplot(2, 5, i+6)
    plt.imshow(X_test_noisy_dl[i].reshape(28, 28), cmap='gray')
    
    # Prédictions des deux modèles
    rf_pred = y_pred_rf_noisy[i]
    dl_pred = np.argmax(model.predict(X_test_noisy_dl[i:i+1], verbose=0))
    
    color = 'green' if rf_pred == y_test_subset[i] and dl_pred == y_test_subset[i] else 'red'
    plt.title(f"RF: {rf_pred} | DL: {dl_pred}", color=color)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Partie 5: Comparaison visuelle des performances
print("\n## 5. Comparaison des approches")

# Tableau récapitulatif
comparison_data = {
    'Aspect': ['Prétraitement', 'Extraction de caractéristiques', 'Temps d\'entraînement (s)', 
               'Précision standard (%)', 'Précision avec bruit (%)', 'Baisse de performance (%)'],
    'Machine Learning (Random Forest)': [
        'Complexe (aplatissement, normalisation, PCA)',
        'Manuelle (PCA)',
        f'{ml_training_time:.2f}',
        f'{accuracy_rf*100:.2f}',
        f'{accuracy_rf_noisy*100:.2f}',
        f'{(accuracy_rf - accuracy_rf_noisy)*100:.2f}'
    ],
    'Deep Learning (CNN)': [
        'Simple (normalisation)',
        'Automatique (couches de convolution)',
        f'{dl_training_time:.2f}',
        f'{accuracy_dl*100:.2f}',
        f'{accuracy_dl_noisy*100:.2f}',
        f'{(accuracy_dl - accuracy_dl_noisy)*100:.2f}'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Visualisation graphique des performances
metrics = ['Temps d\'entraînement (s)', 'Précision standard (%)', 'Précision avec bruit (%)']
ml_values = [ml_training_time, accuracy_rf*100, accuracy_rf_noisy*100]
dl_values = [dl_training_time, accuracy_dl*100, accuracy_dl_noisy*100]

plt.figure(figsize=(12, 6))
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, ml_values, width, label='Machine Learning (RF)')
plt.bar(x + width/2, dl_values, width, label='Deep Learning (CNN)')

plt.ylabel('Valeurs')
plt.title('Comparaison ML vs DL')
plt.xticks(x, metrics)
plt.legend()

plt.tight_layout()
plt.show()

# Partie 6: Analyse des erreurs avec matrices de confusion
print("\n## 6. Analyse des erreurs de classification")

# Matrices de confusion
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
cm_rf = confusion_matrix(y_test_subset, y_pred_rf_noisy)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de confusion - Random Forest')
plt.xlabel('Prédit')
plt.ylabel('Réel')

plt.subplot(1, 2, 2)
y_pred_dl = np.argmax(model.predict(X_test_noisy_dl, verbose=0), axis=1)
cm_dl = confusion_matrix(y_test_subset, y_pred_dl)
sns.heatmap(cm_dl, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de confusion - CNN')
plt.xlabel('Prédit')
plt.ylabel('Réel')

plt.tight_layout()
plt.show()

# Conclusion
print("\n## 7. Conclusion et observations clés")
print("""
Observations clés sur les différences entre Machine Learning classique et Deep Learning:

1. **Prétraitement des données**:
   - ML classique: Nécessite un prétraitement complexe (aplatissement, normalisation, réduction de dimension)
   - Deep Learning: Prétraitement plus simple (normalisation basique)

2. **Extraction de caractéristiques**:
   - ML classique: Extraction manuelle de caractéristiques (ici via PCA)
   - Deep Learning: Apprentissage automatique des caractéristiques via les couches de convolution

3. **Performances**:
   - Deep Learning: Généralement plus performant sur les données brutes à haute dimension
   - ML classique: Peut être efficace après une bonne ingénierie de caractéristiques

4. **Robustesse**:
   - Deep Learning: Souvent plus robuste face aux perturbations et variations
   - ML classique: Plus sensible aux modifications des entrées

5. **Temps et ressources**:
   - Deep Learning: Peut nécessiter plus de temps d'entraînement et de ressources
   - ML classique: Généralement plus rapide à entraîner sur des datasets de taille modérée

Ces observations vous aideront à choisir l'approche la plus adaptée selon le contexte de votre projet de développement.
""")

print("\n## Pour aller plus loin")
print("""
Suggestions pour approfondir votre compréhension:
1. Testez d'autres types de perturbations (rotation, zoom, occlusion)
2. Comparez différents algorithmes de ML classique (SVM, KNN, etc.)
3. Expérimentez avec des architectures CNN plus complexes
4. Essayez ces approches sur d'autres types de données (texte, séries temporelles)
""")